In [1]:
from pyomo.environ import *

# Model
model = ConcreteModel()

# Sets
model.projects = Set(initialize=['p1', 'p2', 'p3','p4'])
model.engineers = Set(initialize=['e1','e2','e3'])

# Data
designer_score = {('e1','p1'):90,('e1','p2'):80,('e1','p3'):10,('e1','p4'):50,('e2','p1'):60,('e2','p2'):70,('e2','p3'):50,('e2','p4'):65,
                      ('e3','p1'):70,('e3','p2'):40,('e3','p3'):80,('e3','p4'):85}
required_hours = {'p1':70,'p2':50,'p3':85,'p4':35}

# Parameters
model.designer_score = Param(model.engineers,model.projects,initialize=designer_score)
model.required_hours = Param(model.projects, initialize = required_hours)

# Decision Variables
model.x = Var(model.engineers,model.projects, domain=NonNegativeReals)

#Constraint
def project_requirement(model, project):
    return sum(model.x[i,project] 
                for i in model.engineers) == model.required_hours[project]

model.ProjectConstraints = Constraint(model.projects, rule=project_requirement)
    
def min_hours(model, engineers):
    return sum(model.x[engineers, i]
               for i in model.projects) <= 80
    
model.TimeConstraint = Constraint(model.engineers,rule=min_hours)

# Objective Function
model.total_value = Objective(expr=
    sum(model.designer_score[i,j] * model.x[i,j] for i in model.engineers for j in model.projects), 
    sense=maximize)   

# Solve the model using a solver
SolverFactory('gurobi').solve(model)

# Print the solution
for i in model.engineers:
    for j in model.projects:
        print(f"{i,j}: {model.x[i,j].value}")

# Print the total cost
print('\n',f"Objective Value: {model.total_value.expr()}",'\n')

('e1', 'p1'): 70.0
('e1', 'p2'): 10.0
('e1', 'p3'): 0.0
('e1', 'p4'): 0.0
('e2', 'p1'): 0.0
('e2', 'p2'): 40.0
('e2', 'p3'): 5.0
('e2', 'p4'): 35.0
('e3', 'p1'): 0.0
('e3', 'p2'): 0.0
('e3', 'p3'): 80.0
('e3', 'p4'): 0.0

 Objective Value: 18825.0 

